In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import torch
import torch.optim as optim
import torch.nn as nn
import datetime
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
import torch.nn.functional as F
# from data_generation.image_classication import generate_dataset
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [2]:
class StepByStep(object):
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)
        self.losses = []
        self.val_loss = []
        self.epochs = 0
        self.train_step_fn = self.train_step_fn()
        self.val_step_fn = self.val_step_fn()
    def to(self, device):
        try:
            self.device = device

            self.model.to(self.device)

        except RuntimeError:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            print("can't send it to ${device}, sending to ${self.device} instead")
            self.model.to(self.device)
            self.train_loader = None
            self.valid_loader = None
            self.writer = None
    def set_loaders(self, train_loader, val_loader = None):
        self.train_loader = train_loader
        self.valid_loader = val_loader
    def set_tensorboard(self, name, folder='runs'):
        suffix = datetime.datetime.now().strftime('%Y%d%H%M%S')
        self.writer = SummaryWriter(f'{folder}/{name}_{suffix}')

In [3]:
#NCHW:: Number of sample : Channel : Height : Width
single = np.array([[[[5, 0, 8, 7, 8, 1],
                     [1, 9, 5, 0, 7, 7],
                     [6, 0, 2, 4, 6, 6],
                     [9, 7, 6, 6, 8, 4],
                     [8, 3, 8, 5, 1, 3],
                     [7, 2, 7, 0, 1, 0]]]])
single.shape

(1, 1, 6, 6)

In [4]:
identity = np.array([[[[0, 0, 0],
                       [0, 1, 0],
                       [0, 0, 0]]]])
identity.shape

(1, 1, 3, 3)

In [6]:
region = single[:, :, 0:3, 0:3]
print('region::', region)
filter_region = region * identity
print('after element-wise multiplication:: ', filter_region)
total = filter_region.sum()
total
print('total::', total)

region:: [[[[5 0 8]
   [1 9 5]
   [6 0 2]]]]
after element-wise multiplication::  [[[[0 0 0]
   [0 9 0]
   [0 0 0]]]]
total:: 9


In [8]:
image = torch.as_tensor(single).float()
kernel_identity = torch.as_tensor(identity).float()

In [9]:
convolved = F.conv2d(image, kernel_identity, stride=1)
convolved

tensor([[[[9., 5., 0., 7.],
          [0., 2., 4., 6.],
          [7., 6., 6., 8.],
          [3., 8., 5., 1.]]]])

In [10]:
convo = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1)(image)
convo

tensor([[[[ 0.4430,  3.6010,  1.0918,  2.1196],
          [ 2.6515, -0.0436,  0.5682,  2.6184],
          [ 2.6814,  2.3326,  1.8248,  2.7297],
          [ 2.6673, -0.2268,  1.2149,  0.7308]]]],
       grad_fn=<ConvolutionBackward0>)

In [11]:
conv_multiple = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1)(image)
conv_multiple

tensor([[[[-2.4775,  0.3060, -1.8897, -0.4312],
          [-0.2428, -0.7374, -2.3525, -0.1082],
          [-0.5334,  0.9730, -0.0105, -0.7030],
          [-1.7993,  0.8285, -0.7703,  0.1730]],

         [[ 0.4435, -2.5179, -2.1540, -0.0429],
          [-0.8305,  1.3972,  1.5624, -0.4722],
          [ 0.3975,  0.3355, -0.9774, -1.5948],
          [-2.4740,  0.2556, -1.6433, -2.7288]]]],
       grad_fn=<ConvolutionBackward0>)

In [13]:
convolved_stride2 = F.conv2d(image, kernel_identity, stride=2)
convolved_stride2

tensor([[[[9., 0.],
          [7., 6.]]]])

In [14]:
constant_padder = nn.ConstantPad2d(padding=1, value=0)
constant_padder(image)

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 5., 0., 8., 7., 8., 1., 0.],
          [0., 1., 9., 5., 0., 7., 7., 0.],
          [0., 6., 0., 2., 4., 6., 6., 0.],
          [0., 9., 7., 6., 6., 8., 4., 0.],
          [0., 8., 3., 8., 5., 1., 3., 0.],
          [0., 7., 2., 7., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]]]])

In [17]:
padded = F.pad(image, pad=(1, 1, 1, 1), mode="constant", value=0)
padded

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 5., 0., 8., 7., 8., 1., 0.],
          [0., 1., 9., 5., 0., 7., 7., 0.],
          [0., 6., 0., 2., 4., 6., 6., 0.],
          [0., 9., 7., 6., 6., 8., 4., 0.],
          [0., 8., 3., 8., 5., 1., 3., 0.],
          [0., 7., 2., 7., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]]]])